# Exemplo real aplicando o algoritmo Funk SVD CF - Base: Banco

In [5]:
import numpy as np
import pandas as pd

%matplotlib inline

In [6]:
# !conda install -c conda-forge lenskit

In [7]:
data = pd.read_excel("Base_IAP.xlsx")

In [9]:
data = data[(data['Sigla Tipo Pessoa'] == 'PF') & (data[ 'Valor Saldo Cliente Conta Capital Diário'] != 0)]

In [10]:
data['Atividade Econômica'].unique()

array(['PESSOA FISICA'], dtype=object)

In [11]:
data.columns

Index(['Número CPF/CNPJ', 'Atividade Econômica', 'CNAE',
       'Data Início Relacionamento', 'Descrição Categoria Produtor Rural',
       'Escolaridade', 'Estado Civil', 'Idade', 'Profissão',
       'Renda Bruta Mensal', 'Sexo', 'Sigla Tipo Pessoa', 'Tipo Renda',
       'Valor Bem Imóvel', 'Valor Bem Móvel', 'Nome PA',
       'Valor Saldo Cliente Conta Capital Diário',
       'Quantidade Contratos Crédito',
       'Valor Limite Conta Cartão Contratado Diário Cliente Cartão',
       'Quantidade Cota Ativa Cliente Consórcio',
       'Valor Saldo Final Cliente Poupança', 'Valor Saldo Diário Cliente LCA',
       'Valor Saldo Diário Cliente RDC', 'Valor Saldo Diário Cliente LCI',
       'Quantidade Seguro Vigente', 'Valor Contribuição Cliente Previ'],
      dtype='object')

In [12]:
dfprod = data[['Quantidade Contratos Crédito',
       'Valor Limite Conta Cartão Contratado Diário Cliente Cartão',
       'Valor Saldo Final Cliente Poupança',
       'Valor Saldo Diário Cliente RDC',
       'Quantidade Seguro Vigente']]

In [14]:
dfprod.shape[1]

5

In [15]:
dfprod.columns

Index(['Quantidade Contratos Crédito',
       'Valor Limite Conta Cartão Contratado Diário Cliente Cartão',
       'Valor Saldo Final Cliente Poupança', 'Valor Saldo Diário Cliente RDC',
       'Quantidade Seguro Vigente'],
      dtype='object')

In [16]:
dfrename = dfprod.rename(columns={'Quantidade Contratos Crédito':'credito',
                                  'Valor Limite Conta Cartão Contratado Diário Cliente Cartão':'cartao', 
                                  'Valor Saldo Final Cliente Poupança':'poupanca',
                                  'Valor Saldo Diário Cliente RDC':'rdc',
                                  'Quantidade Seguro Vigente':'seguro',                              
                                 }, inplace=False)

In [17]:
dfrename.columns

Index(['credito', 'cartao', 'poupanca', 'rdc', 'seguro'], dtype='object')

In [18]:
for i in dfrename.columns:
    dfrename.loc[dfrename[i]!=0,i] = 1
       

In [19]:
dfrename

,credito,cartao,poupanca,rdc,seguro
5,0,1,0.0,1.0,0
6,0,0,0.0,0.0,0
8,0,0,0.0,0.0,0
9,0,0,0.0,0.0,0
10,0,1,0.0,1.0,0
...,...,...,...,...,...
111392,0,0,0.0,0.0,0
111393,0,0,0.0,0.0,0
111394,0,0,0.0,0.0,0
111395,1,1,0.0,0.0,0


### Reorganizando a tabela para inserir na Biblioteca Surprise

In [20]:
index = dfrename.index
columns = dfrename.columns
items = ['credito', 'cartao', 'poupanca','rdc','seguro']*47205
items = pd.Series(items)

lista_index = []
for i in index:
    lista_index.append(i)
    lista_index.append(i)
    lista_index.append(i)
    lista_index.append(i)
    lista_index.append(i)

lista_index = pd.Series(lista_index)

lista_compras = []
for i,j in dfrename.iterrows():
    lista_compras.append(j)
    
lista_series = [] 
for i in lista_compras:
    lista_series.append(pd.Series(i))
        
lista_series = pd.Series(lista_series)

concat_compras = []
for i in lista_series:
    concat_compras.append(i.values)

concat_compras = [l.tolist() for l in concat_compras]
concat_compras = pd.Series(list(concat_compras))

ls = []
  
for i in concat_compras:
    ls.extend(i)

concat_compras = pd.Series(ls)

df = pd.concat([lista_index,items,concat_compras], axis=1)
df.rename(columns={0: "user",1:"item", 2:"raw_ratings"}, inplace=True)
df.head(20)

,user,item,raw_ratings
0,5,credito,0.0
1,5,cartao,1.0
2,5,poupanca,0.0
3,5,rdc,1.0
4,5,seguro,0.0
5,6,credito,0.0
6,6,cartao,0.0
7,6,poupanca,0.0
8,6,rdc,0.0
9,6,seguro,0.0


In [21]:
# Salvando a tabela gerada em um arquivo.csv

# df.to_csv('original_ratings_base_iap')

In [22]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import pandas as pd

from surprise import SVD
from surprise import Reader

# You'll need to create a dummy reader
reader = Reader(line_format='user item rating', rating_scale=(0, 1))

# Also, a dummy Dataset class
class MyDataset(dataset.DatasetAutoFolds):

    def __init__(self, df, reader):

        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in
                            zip(df['user'], df['item'], df['raw_ratings'])]
        self.reader=reader

data = MyDataset(df, reader)

## Separando os dados em teste e treinamento 3 vezes. Aplicado KFold=3

In [23]:
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import KFold

# Load the movielens-100k dataset

# define a cross-validation iterator
kf = KFold(n_splits=3)

algo = SVD()
all_pred = []
for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    pred = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(pred, verbose=True)
    all_pred.append(pred)

RMSE: 0.4106
RMSE: 0.4109
RMSE: 0.4106


In [24]:
all_pred

[[Prediction(uid=6203, iid='seguro', r_ui=0.0, est=0.13110728326218188, details={'was_impossible': False}),
  Prediction(uid=11491, iid='credito', r_ui=0.0, est=0.10432854233883032, details={'was_impossible': False}),
  Prediction(uid=3764, iid='seguro', r_ui=0.0, est=0.21220501975910394, details={'was_impossible': False}),
  Prediction(uid=6342, iid='seguro', r_ui=0.0, est=0.19440331054237625, details={'was_impossible': False}),
  Prediction(uid=87567, iid='cartao', r_ui=0.0, est=0.522139896046219, details={'was_impossible': False}),
  Prediction(uid=80359, iid='cartao', r_ui=0.0, est=0.49886764289042834, details={'was_impossible': False}),
  Prediction(uid=72283, iid='poupanca', r_ui=0.0, est=0.3031816443431298, details={'was_impossible': False}),
  Prediction(uid=50478, iid='seguro', r_ui=0.0, est=0.17468683143706515, details={'was_impossible': False}),
  Prediction(uid=111085, iid='credito', r_ui=0.0, est=0.17331373362092603, details={'was_impossible': False}),
  Prediction(uid=692

In [25]:
import numpy as np

all_pred = np.array(all_pred)
all_pred.shape

(3, 78675, 5)

In [28]:
cliente = []
for i in all_pred[0]:
    cliente.append(i[0])
for i in all_pred[1]:
    cliente.append(i[0])
for i in all_pred[2]:
    cliente.append(i[0])
    
produto= []
for i in all_pred[0]:
    produto.append(i[1])
for i in all_pred[1]:
    produto.append(i[1])
for i in all_pred[2]:
    produto.append(i[1])
    
rec= []
for i in all_pred[0]:
    rec.append(i[3])
for i in all_pred[1]:
    rec.append(i[3])
for i in all_pred[2]:
    rec.append(i[3])

In [29]:
print(len(cliente))
print(len(produto))
print(len(rec))

236025
236025
236025


#### Recomendaç]oes totais geradas com números de 0 a 1, em que quanto mais próximo de 1, maior a pontuação de recomendação

In [38]:
recs = pd.DataFrame({'id_cliente':cliente,'produto':produto,'recomendacao_funk_svd':rec})
recs.sort_values(by='recomendacao_funk_svd', ascending=False).head(60)

,id_cliente,produto,recomendacao_funk_svd
7726,76621,cartao,0.890986
122507,17675,cartao,0.874340
73268,105002,cartao,0.872187
54634,77328,cartao,0.868946
58150,38583,cartao,0.864289
38218,43874,cartao,0.860236
10728,60237,cartao,0.848710
29964,81896,cartao,0.844894
138831,18588,cartao,0.843262
123797,109140,cartao,0.842540


#### Recomendações para Crédito Pessoal

In [39]:
rec_credito = recs[recs['produto'] == 'credito'].sort_values(by='recomendacao_funk_svd',ascending=False)
rec_credito

,id_cliente,produto,recomendacao_funk_svd
17852,77580,credito,0.502321
61505,36115,credito,0.498466
136700,27039,credito,0.491285
2817,5165,credito,0.490525
12559,65614,credito,0.483940
...,...,...,...
87485,93548,credito,0.070382
94058,74111,credito,0.069160
82230,83837,credito,0.069070
18326,103270,credito,0.067706


#### Recomendações para Cartão de Crédito

In [40]:
rec_cartao = recs[recs['produto'] == 'cartao'].sort_values(by='recomendacao_funk_svd',ascending=False)
rec_cartao

,id_cliente,produto,recomendacao_funk_svd
7726,76621,cartao,0.890986
122507,17675,cartao,0.874340
73268,105002,cartao,0.872187
54634,77328,cartao,0.868946
58150,38583,cartao,0.864289
...,...,...,...
233838,66526,cartao,0.431262
101173,102441,cartao,0.430510
232151,51507,cartao,0.429297
169929,22728,cartao,0.426799


#### Recomendações para Poupança

In [41]:
rec_poupanca = recs[recs['produto'] == 'poupanca'].sort_values(by='recomendacao_funk_svd',ascending=False)
rec_poupanca


,id_cliente,produto,recomendacao_funk_svd
31250,10863,poupanca,0.463038
53929,6283,poupanca,0.455501
70846,15096,poupanca,0.453953
123270,39302,poupanca,0.453375
56124,56358,poupanca,0.453192
...,...,...,...
214379,41560,poupanca,0.037434
207829,60704,poupanca,0.035513
191613,91821,poupanca,0.032933
192201,80909,poupanca,0.029181


#### Recomendações para Seguro

In [42]:
rec_seguro = recs[recs['produto'] == 'seguro'].sort_values(by='recomendacao_funk_svd',ascending=False)
rec_seguro

,id_cliente,produto,recomendacao_funk_svd
67655,42873,seguro,0.473809
43107,99855,seguro,0.469353
56526,67466,seguro,0.464451
72634,48809,seguro,0.462804
34742,66349,seguro,0.458164
...,...,...,...
111538,11868,seguro,0.040222
101556,66145,seguro,0.039482
105511,39653,seguro,0.037323
156058,16157,seguro,0.035419


#### Recomendações para RDC

In [43]:
rec_rdc = recs[recs['produto'] == 'rdc'].sort_values(by='recomendacao_funk_svd',ascending=False)
rec_rdc

,id_cliente,produto,recomendacao_funk_svd
82001,81636,rdc,0.474633
103139,9291,rdc,0.463653
114085,80540,rdc,0.462881
155149,83876,rdc,0.462196
91848,62280,rdc,0.462146
...,...,...,...
7068,85528,rdc,0.046651
230085,18169,rdc,0.045520
10305,23189,rdc,0.043400
62337,73728,rdc,0.040405


In [3]:
#SALVANDO RECOMENDAÇÕES

# recs.to_csv('rec_funk_svd.csv')
# recs.to_excel('rec_funk_svd.xlsx')
# rec_cartao.to_excel('rec_cartao.xlsx')
# rec_poupanca.to_excel('rec_poupanca.xlsx')
# rec_seguro.to_excel('rec_seguro.xlsx')
# rec_credito.to_excel('rec_credito.xlsx')
# rec_rdc.to_excel('rec_rdc.xlsx')